Utility to plot planet positions for a date range.
Based on info https://ssd.jpl.nasa.gov/?planet_pos and https://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf

In [15]:
import math
import numpy as np
import pandas as pd
import datetime
import time
import glob
from astropy.time import Time
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## borrowed from Paul Schlyter's tutorial - regular math.atan doesnt have these properties
def arctan_stleimlen(x,y):
    if x==0:
        if y==0:
            return 0
        elif y>0:
            return 90
        else:
            return -90
    elif x>0:
        return math.atan(y/x)*180/math.pi
    else: ## x<0
        if y>=0:
            return math.atan(y/x)*180/math.pi + 180
        else:
            return math.atan(y/x)*180/math.pi - 180
        
def _arctan_stleimlen(x,y):
    ans = arctan_stleimlen(x,y)
    print (x,y, ans)
    return ans        

##  Vector shorthands for sin, cos, tan ...
_cos = lambda x: x.apply (lambda y: math.cos(y*math.pi/180))
_sin = lambda x: x.apply (lambda y: math.sin(y*math.pi/180))
_atan = lambda x: x.apply (lambda y: math.atan(y)*180/math.pi)
_sqrt = lambda x: x.apply (lambda y: math.sqrt(y))
_dist = lambda x,y,z,L=2 : (x**L+y**L+z**L)**(1/L)

_atan2 = lambda y,x: pd.Series(list(zip(y,x)), index=(y.index)).apply (lambda e: ((arctan_stleimlen(e[1], e[0])) +360) %360 ) #.reindex(index=y.index)
_atan1 = lambda y,x: (y/x).apply (lambda e: math.atan(e)*180/math.pi)


In [3]:
# From http://www.stjarnhimlen.se/comp/ppcomp.html - this method validates with Stellarium ( better than Standish )
PP2 = pd.read_csv('planet_params_stjarmhimlen.csv' ).set_index('planet')
PP2['L'] = PP2['M'] + PP2['w'] + PP2['N']
PP2['L_d'] = PP2['M_d'] + PP2['w_d'] + PP2['N_d']
PP2
PP2_Rate = PP2.loc[:, ['a_d', 'e_d', 'i_d', 'L_d', 'M_d', 'N_d' , 'w_d'] ]
PP2_Base = PP2.loc[:, ['a', 'e', 'i', 'L', 'M', 'N' , 'w'] ]
pd.options.display.float_format = '{:,.6f}'.format
#display(PP2_Rate)
#display(PP2_Base)

In [4]:
JD_BCE_3000_JAN_1   = 625332.5  # -3000-01-01T00:00:00.000
JD_JEPPY_KALI_YUGA  = 588465.5  # -3101-01-23T00:00:00.000  ( +25 gets to KY 17/Feb/-3101 )
JD_JEPPY_OFFSET     = 2415020   # 1899-12-31T12:00:00.000

def rev(x):
    rev = x - math.trunc(x/360.0)*360.0
    if rev<0.0:
        rev=rev+360.0      
    return rev

def solve_E(M, e):
    TOLERANCE = 1E-6
    e_deg = e*180/math.pi
    
    ## E_ is in degree
    E_ = M + e_deg*_sin(M)
    
    for n in range(1,500):
        delta_M = M - (E_ - e_deg*_sin(E_))
        delta_E = delta_M/(1 - e*_cos(E_))
        if (delta_E.abs() < TOLERANCE).all() :
            ## print(f'converged after {n} iterations')
            break
        E_ = E_ + delta_E
        
    return E_

def get_planet_pos_starjm (jd = JD_BCE_3000_JAN_1) :
    isot = Time(jd, format='jd').isot
    #display(jd, isot )
    T = ( jd - 2451543.5) #/36525
    params = PP2_Base + PP2_Rate.values*T
    
    params['M'] = params['M'].apply( lambda x : rev(x))
    waileo = params
    e_star = waileo['e'] * 180/math.pi
    #display(waileo)
    
    En = solve_E(waileo['M'], waileo['e'])

    x_prime = waileo['a']*(_cos(En) - waileo['e'])
    y_prime = waileo['a']*_sin(En)* _sqrt( 1 - waileo['e'] *waileo['e'] )
    v_nu = _atan2(y_prime, x_prime)
    r = _sqrt ( y_prime*y_prime + x_prime*x_prime)
    
    lon = (v_nu+ waileo['w'])%360
    N=waileo['N']
    I=waileo['i']
    
    ## ecliptic rectangular, also called xh,yh,zh
    x_eclip = r * ( _cos(N) * _cos(lon) - _sin(N) * _sin(lon) * _cos(I) )
    y_eclip = r * ( _sin(N) * _cos(lon) + _cos(N) * _sin(lon) * _cos(I) )
    z_eclip = r * _sin(lon) * _sin(I)   
    
    geoc_x = x_eclip + x_eclip['Sun']
    geoc_y = y_eclip + y_eclip['Sun']
    geoc_z = z_eclip + z_eclip['Sun']
    
    for obj in ['Moon', 'Sun'] :
        geoc_x[obj] = x_eclip[obj]
        geoc_y[obj] = y_eclip[obj]
        geoc_z[obj] = z_eclip[obj]
        
    geo_r = _dist(geoc_x, geoc_y, geoc_z)
        
    #ecl_long0 = _atan( geoc_y/ geoc_x)
    #ecl_long1 = _atan1( geoc_y, geoc_x)
    #ecl_long2 = _atan2( geoc_y, geoc_x)
    ecl_long = _atan2( geoc_y, geoc_x)
    ecl_lati = _atan1( geoc_z, _sqrt ( geoc_y**2 + geoc_x**2 ) )
    
    ans = pd.DataFrame ({
        'jd' : ecl_long.apply(lambda v : jd )
        , 'date' : ecl_long.apply(lambda v : isot ) 
        ,'ecl_long' : ecl_long
        ,'ecl_lati' : ecl_lati
        ,'geo_r'  : geo_r
        ,'geoc_x' : geoc_x
        ,'geoc_y' : geoc_y
        ,'geoc_z' : geoc_z
        ,'r': r

    })
    '''
        ,'x_eclip': x_eclip
        ,'y_eclip': y_eclip
        ,'z_eclip': z_eclip
        ,'En' : En
        ,'M_check': waileo['M'] - (En - e_star*_sin(En))
        ,'x_prime': x_prime
        ,'y_prime': y_prime
        ,'v_nu'   : v_nu
        ,'N'      : N
        ,'lon'    : lon
        ,'I'      : I
    '''
    
    #display(ans)
    return ans

In [14]:
def save_snapshot (tag, acc, f_cnt) :
    if not len(acc) :
        return
    
    ans = pd.DataFrame().append(acc)
    fn = tag + ".csv"
    if (f_cnt < 10 and f_cnt %1 == 0) or (f_cnt < 100 and f_cnt %1 == 0) or (f_cnt % 100 == 0) :
        print ( '%6.2f secs, %5d files, %s %s %s' % (  time.time() - begin_time, f_cnt, "Saving" , fn,  str(ans.shape) ))
    ans.to_csv(fn)
    

acc = []
pyear = None
pcentury = None
begin_time = time.time()
f_cnt = 1 
for jd in range ( 0, 365*5000, 7 ) :
    t = JD_BCE_3000_JAN_1 + jd*1 
    #t = (1720335.5+7) + jd*1  # ran till here -2-01-07T00:00:00.000 , died when entered CE due to bad regex see below 
    tstr =  Time(t, format='jd').isot
    year = re.match("^(.?\d+)\-", tstr)[1]  # added ? to fix CE parse bug
    n = int (year)
    century = ('%03d00' if n<0 else '%02d00') % (n//100) 
    century = 'c_' + century
    if not pyear:
        pyear = year
        pcentury = century
        
    if pcentury != century :
        save_snapshot (pcentury, acc, f_cnt)
        f_cnt = f_cnt +1
        acc = []
        
    if (( jd < 365*140 and jd % 365 == 0 ) ) :
        print ( '%6.2f secs, %5d years  - %s  : %s'  % (  time.time() - begin_time, jd //365,  tstr , pcentury))
        
    df = get_planet_pos_starjm(t) #2453005.458333 1539168.9791665
    acc.append(df)
    pcentury = century

save_snapshot (pcentury, acc, f_cnt)


  0.00 secs,     0 years  - -3000-01-01T00:00:00.000  : c_-3000
  4.34 secs,     7 years  - -2994-12-31T00:00:00.000  : c_-3000
  8.49 secs,    14 years  - -2987-12-29T00:00:00.000  : c_-3000
 12.60 secs,    21 years  - -2980-12-27T00:00:00.000  : c_-3000
 16.70 secs,    28 years  - -2973-12-26T00:00:00.000  : c_-3000
 20.76 secs,    35 years  - -2966-12-24T00:00:00.000  : c_-3000
 24.85 secs,    42 years  - -2959-12-22T00:00:00.000  : c_-3000
 28.98 secs,    49 years  - -2952-12-20T00:00:00.000  : c_-3000
 33.08 secs,    56 years  - -2945-12-19T00:00:00.000  : c_-3000
 37.26 secs,    63 years  - -2938-12-17T00:00:00.000  : c_-3000
 41.76 secs,    70 years  - -2931-12-15T00:00:00.000  : c_-3000
 45.81 secs,    77 years  - -2924-12-13T00:00:00.000  : c_-3000
 49.95 secs,    84 years  - -2917-12-12T00:00:00.000  : c_-3000
 53.96 secs,    91 years  - -2910-12-10T00:00:00.000  : c_-3000
 58.07 secs,    98 years  - -2903-12-08T00:00:00.000  : c_-3000
 60.49 secs,     1 files, Saving c_-3000

In [30]:
acc = []
for f in glob.glob('c*csv') :
    print(f)
    acc.append(pd.read_csv(f))

c_1700.csv
c_-1100.csv
c_-0800.csv
c_-1300.csv
c_1500.csv
c_-1700.csv
c_1100.csv
c_0800.csv
c_1300.csv
c_-2800.csv
c_-1500.csv
c_1400.csv
c_-1200.csv
c_-1000.csv
c_1600.csv
c_-0900.csv
c_-3000.csv
c_-1400.csv
c_-2900.csv
c_1200.csv
c_1000.csv
c_-1600.csv
c_0900.csv
c_-0300.csv
c_0500.csv
c_-2700.csv
c_-2500.csv
c_-1800.csv
c_0700.csv
c_-0100.csv
c_0300.csv
c_-0500.csv
c_-2100.csv
c_1800.csv
c_-2300.csv
c_-0700.csv
c_0100.csv
c_-1900.csv
c_-2400.csv
c_0600.csv
c_0400.csv
c_-0200.csv
c_-2600.csv
c_-2200.csv
c_1900.csv
c_0000.csv
c_-0600.csv
c_-0400.csv
c_0200.csv
c_-2000.csv


In [60]:
pp_df = pd.DataFrame().append(acc).rename(columns={"r": "helio_r"}).sort_values( by = ['jd', 'planet']).reset_index() 
display(pp_df.shape)

(1825005, 11)

In [59]:
display(pp_df.head(10))
display(pp_df.tail(10))
pp_df.shape


,index,planet,jd,date,ecl_long,ecl_lati,geo_r,geoc_x,geoc_y,geoc_z,helio_r
0,0,Jupiter,"625,332.500000",-3000-01-01T00:00:00.000,147.990549,1.836484,4.657728,-3.947542,2.467603,0.149267,5.388639
1,1,Mars,"625,332.500000",-3000-01-01T00:00:00.000,201.092504,1.111740,1.303462,-1.215902,-0.468995,0.025290,1.497630
2,2,Mercury,"625,332.500000",-3000-01-01T00:00:00.000,280.796766,-1.705000,1.353961,0.253520,-1.329404,-0.040285,0.357952
3,3,Moon,"625,332.500000",-3000-01-01T00:00:00.000,82.790020,5.139114,57.874429,7.234392,57.186001,5.184058,57.874429
4,4,Saturn,"625,332.500000",-3000-01-01T00:00:00.000,82.174335,0.727898,8.543668,1.163206,8.463417,0.108538,9.492953
5,5,Sun,"625,332.500000",-3000-01-01T00:00:00.000,281.092224,-0.000000,0.997735,0.191953,-0.979096,-0.000000,0.997735
6,6,Venus,"625,332.500000",-3000-01-01T00:00:00.000,313.164930,-0.933469,1.332787,0.911640,-0.971989,-0.021713,0.720049
7,7,Jupiter,"625,339.500000",-3000-01-08T00:00:00.000,147.549308,1.867497,4.579138,-3.862069,2.455739,0.149226,5.387679
8,8,Mars,"625,339.500000",-3000-01-08T00:00:00.000,205.624555,1.027646,1.243745,-1.121239,-0.537799,0.022306,1.489018
9,9,Mercury,"625,339.500000",-3000-01-08T00:00:00.000,294.859032,-0.929267,1.292205,0.543155,-1.172322,-0.020957,0.322779


,index,planet,jd,date,ecl_long,ecl_lati,geo_r,geoc_x,geoc_y,geoc_z,helio_r
1824995,35305,Saturn,"2,450,323.500000",1996-08-28T00:00:00.000,5.900914,-2.562599,8.647240,8.592818,0.888120,-0.386626,9.527096
1824996,35306,Sun,"2,450,323.500000",1996-08-28T00:00:00.000,154.975127,0.000000,1.010091,-0.915268,0.427280,0.000000,1.010091
1824997,35307,Venus,"2,450,323.500000",1996-08-28T00:00:00.000,109.399486,-2.536755,0.768832,-0.255119,0.724472,-0.034029,0.724760
1824998,35308,Jupiter,"2,450,330.500000",1996-09-04T00:00:00.000,277.772048,-0.187656,4.652234,0.629129,-4.609474,-0.015237,5.174652
1824999,35309,Mars,"2,450,330.500000",1996-09-04T00:00:00.000,116.331226,0.878491,2.101342,-0.931961,1.883095,0.032218,1.567791
1825000,35310,Mercury,"2,450,330.500000",1996-09-04T00:00:00.000,183.491496,-4.024507,0.726077,-0.722942,-0.044109,-0.050958,0.432752
1825001,35311,Moon,"2,450,330.500000",1996-09-04T00:00:00.000,61.337617,-4.057761,61.423997,29.388013,53.762070,-4.346492,61.423997
1825002,35312,Saturn,"2,450,330.500000",1996-09-04T00:00:00.000,5.452002,-2.581973,8.593785,8.546223,0.815683,-0.387139,9.524924
1825003,35313,Sun,"2,450,330.500000",1996-09-04T00:00:00.000,161.747513,0.000000,1.008434,-0.957695,0.315847,0.000000,1.008434
1825004,35314,Venus,"2,450,330.500000",1996-09-04T00:00:00.000,116.613443,-1.969582,0.822656,-0.368307,0.735060,-0.028274,0.723822


(1825005, 11)

In [74]:
pp_df.to_csv("./planets_positions_from_3000BCE_for_5k_years_at_7day_interval.csv", index=None)
#head helio_long.csv

In [75]:
!ls -hal planets*csv 


-rw-r--r--  1 sunder.chakravarty  staff   305M 13 Jan 14:37 planets_positions_from_3000BCE_for_5k_years_at_7day_interval.csv


In [ ]:
#[Time(t, format='jd').unix for t in [JD_BCE_3000_JAN_1, JD_JEPPY_KALI_YUGA, JD_JEPPY_OFFSET,  JD_JEPPY_OFFSET + 365*120+18.75] ]

In [ ]:
#[ (  n , ('%05d' if n<0 else '%04d') % (n//1000) )for n in range(-3000,3001, 80)]

In [ ]:
# Standish -  Unable to get this to work
#http://www.met.rdg.ac.uk/~ross/Astronomy/Planets.html - for earth
#https://ssd.jpl.nasa.gov/txt/aprx_pos_planets.pdf for rest
planet_params = pd.read_csv('planet_params.csv' ).set_index('planet')
planet_params2 = pd.read_csv('planet_params2.csv' ).set_index('planet')
PP = pd.merge ( planet_params, planet_params2 , how='left' , on ='planet')
PP=PP.drop ( ['Uranus','Neptune','Pluto'] )
WAILEO  = ['a', 'e', 'I', 'L', 'w', 'O']
WAILEO_CY = [ x + '_cy' for x in WAILEO]
BCSF = ['b','c','s','f']
bcsf =  PP.loc[ :, BCSF]


In [ ]:
def get_planet_pos_standish (jd = JD_BCE_3000_JAN_1) :
    
    display(jd, Time(jd, format='jd').isot)
    T = ( jd - 2451545.0)/36525
    waileo = PP.loc[ :, WAILEO] + PP.loc[ :, WAILEO_CY].values*T
    #display(waileo)

    #=======================
    waileo["w-O"] = waileo['w'] - waileo['O'] # perihi

    x = ( #mean_anomaly
        waileo['L'] - waileo['w'] 
        + bcsf['b']*T*T 
        + bcsf['c'] * _cos(bcsf['f']*T) 
        + bcsf['s'] * _sin(bcsf['f']*T) 
    ) 
    waileo["M"]= ((np.sign(x)*x)%180.0)*np.sign(x)

    e_star = waileo['e'] * 180/math.pi

    waileo['E0'] = waileo['M'] + e_star*_sin(waileo['M'])
    #display(waileo)

    #----------------
    M  = waileo["M"]
    En = waileo['E0']
    TOLERANCE = 1E-6
    for n in range(1,500):
        #deltaM = M - ( En - [e_star*math.sin(x) for x in En*math.pi/180] )
        #deltaE = deltaM /(1 - math.e*np.array([math.cos(x) for x in En*math.pi/180]) )

        deltaM = M - ( En - e_star*_sin(En) )
        deltaE = deltaM /(1 - waileo['e']*_cos(En))   
        if (deltaE.abs() < TOLERANCE).all() :
            waileo['E'] = En
            break 
        En = En + deltaE

    waileo['M_check'] = En - e_star*_sin(En)
    x_prime = waileo['a']*(_cos(En) - waileo['e'])
    y_prime = waileo['a']*_sin(En)* _sqrt( 1 - waileo['e'] *waileo['e'] )
    pd.DataFrame ({ 'x_prime': x_prime, 'y_prime' : y_prime})
    waileo['x_prime'] = x_prime
    waileo['y_prime'] = y_prime
    #display(waileo)

    #===========
    w,O,I =waileo['w-O'], waileo['O'], waileo['I']
    #display (w,O,I)

    sur_x = (_cos(w)*_cos(O) - _sin(w)*_sin(O)*_cos(I))*x_prime + ( -_sin(w)*_cos(O) - _cos(w)*_sin(O)*_cos(I)) *y_prime
    sur_y = (_cos(w)*_sin(O) + _sin(w)*_cos(O)*_cos(I))*x_prime + ( -_sin(w)*_sin(O) + _cos(w)*_cos(O)*_cos(I)) *y_prime
    sur_z = _sin(w)*_sin(I)*x_prime + _cos(w)*_sin(I)*y_prime
    
    #wrt= 'EM_Bary'
    wrt = 'Earth'
    
    bhu_x = sur_x - sur_x[wrt]
    bhu_y = sur_y - sur_y[wrt]
    bhu_z = sur_z - sur_z[wrt]
    
    sur_long = _atan(sur_y/sur_x)
    sur_lati = _atan(sur_z/_sqrt(sur_x**2 + sur_y**2 ))  
    #sur_long = _atan1(sur_y, sur_x)
    #sur_lati = _atan1(sur_z, _sqrt(sur_x**2 + sur_y**2 ))
    #sur_long = _atan2(sur_y,sur_x)
    #sur_lati = _atan2(sur_z,_sqrt(sur_x**2 + sur_y**2 ))
    sur_dist = _sqrt(sur_x**2 + sur_y**2 + sur_z**2  ) 

    bhu_long = _atan(bhu_y/bhu_x)
    bhu_lati = _atan(bhu_z/_sqrt(bhu_x**2 + bhu_y**2) )    
    #bhu_long = _atan1(bhu_y,bhu_x)
    #bhu_lati = _atan1(bhu_z,_sqrt(bhu_x**2 + bhu_y**2) )    
    bhu_dist = _sqrt(bhu_x**2 + bhu_y**2 + bhu_z**2  ) 
    #display (pd.DataFrame ( {'sur': sur_long} ) )
    #display (pd.DataFrame ( {'bhu': bhu_long} ) )
    
    ans = pd.DataFrame ( {
         'sur_x' : sur_x, 'sur_y' : sur_y, 'sur_z': sur_z,
         'sur_lati' : sur_lati, 'sur_long' : sur_long, 'sur_dist': sur_dist,
         'bhu_x' : bhu_x, 'bhu_y' : bhu_y, 'bhu_z': bhu_z,
         'bhu_lati' : bhu_lati, 'bhu_long' : bhu_long, 'bhu_dist': bhu_dist,
    }) #, columns = ['w', '_w'] )    

    #xdf['_sumsq']  = xdf['_cw']*xdf['_cw'] + xdf['_sw']*xdf['_sw']  
    return ans , waileo


In [ ]:
def test_standish () :
    ans, wlo = get_planet_pos_standish( 
          0*806471.355405 
        + 0*623846.480405 
        + 1*2453005.458333
        + 0*2457005
    )
    #(2453006)
    display(ans)